<a href="https://colab.research.google.com/github/GenoKiller777/AprendizajeSupervisado/blob/main/MLFlow_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalacion de MLFlow
!pip install mlflow

In [2]:
!pip install pyngrok

In [4]:
#Checkear si esta instalado
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [3]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment('AprendiendoMLFlow-RandomForest')

2023/10/25 02:38:04 INFO mlflow.tracking.fluent: Experiment with name 'AprendiendoMLFlow-RandomForest' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/760551064086511538', creation_time=1698201484428, experiment_id='760551064086511538', last_update_time=1698201484428, lifecycle_stage='active', name='AprendiendoMLFlow-RandomForest', tags={}>

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import mlflow
import mlflow.sklearn
import sys
import os

In [6]:
def eval_metrics(actual,pred):
  rmse = np.sqrt(mean_squared_error(actual,pred))
  mae = mean_absolute_error(actual,pred)
  r2 = r2_score(actual,pred)
  return rmse, mae, r2

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv', delimiter = ";")

In [ ]:
data.head()

In [18]:
def  train_model(n_estimators,max_depth,max_features,max_leaf_nodes,ccp_alpha):
  #Divida los datos en conjuntos de entrenamiento y prueba. (0,75, 0,25)
    train, test = train_test_split(data)

  # La columna prevista es "calidad", que es un escalar de [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    with mlflow.start_run():
      regressor = RandomForestRegressor(n_estimators = n_estimators ,
                                        max_depth = max_depth,
                                        max_features = max_features,
                                        max_leaf_nodes = max_leaf_nodes,
                                        ccp_alpha = ccp_alpha ,
                                        random_state = 33)
      regressor.fit(train_x, train_y.values.ravel())

      predicted_qualities = regressor.predict(test_x)

      (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

      print(f"RandomForestRegressor (n_estimators={n_estimators:f}, max_depth={max_depth:f},max_features={max_features:f}, max_leaf_nodes={max_leaf_nodes:f},ccp_alpha={ccp_alpha:f}):")
      print(f" RMSE: {rmse}")
      print(f" MAE: {mae}")
      print(f" R2: {r2}")

      mlflow.log_param("n_estimators", n_estimators)
      mlflow.log_param("max_depth", max_depth)
      mlflow.log_param("max_features", max_features)
      mlflow.log_param("max_leaf_nodes", max_leaf_nodes)
      mlflow.log_param("ccp_alpha", ccp_alpha)
      mlflow.log_metric("rmse", rmse)
      mlflow.log_metric("r2", r2)
      mlflow.log_metric("mae", mae)

      mlflow.sklearn.log_model(regressor,"model")

In [19]:
train_model(500,3,3,3,0)

RandomForestRegressor (n_estimators=500.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.000000):
 RMSE: 0.7091385880038996
 MAE: 0.5609066260907529
 R2: 0.2241863840924656


In [20]:
train_model(1000,3,3,3,0)

RandomForestRegressor (n_estimators=1000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.000000):
 RMSE: 0.6852221910497496
 MAE: 0.5560049507807552
 R2: 0.2523118737099097


In [21]:
train_model(5000,3,3,3,0)

RandomForestRegressor (n_estimators=5000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.000000):
 RMSE: 0.6417722227757701
 MAE: 0.5432876805028073
 R2: 0.2631663564085108


In [22]:
train_model(500,3,3,3,1)

RandomForestRegressor (n_estimators=500.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=1.000000):
 RMSE: 0.7677284568721202
 MAE: 0.658806980817348
 R2: -0.007802483126898352


In [23]:
train_model(1000,3,3,3,1)

RandomForestRegressor (n_estimators=1000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=1.000000):
 RMSE: 0.7744413875341047
 MAE: 0.6536848874061715
 R2: -0.0001095771395807077


In [24]:
train_model(5000,3,3,3,1)

RandomForestRegressor (n_estimators=5000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=1.000000):
 RMSE: 0.8163607064285024
 MAE: 0.6960332927439532
 R2: -0.002936149512984043


In [25]:
train_model(500,3,3,3,0.5)

RandomForestRegressor (n_estimators=500.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.500000):
 RMSE: 0.773872865594895
 MAE: 0.6575554628857379
 R2: -0.004873043506949504


In [26]:
train_model(1000,3,3,3,0.5)

RandomForestRegressor (n_estimators=1000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.500000):
 RMSE: 0.7987660958662954
 MAE: 0.670476601334445
 R2: -0.0030298316388677193


In [27]:
train_model(5000,3,3,3,0.5)

RandomForestRegressor (n_estimators=5000.000000, max_depth=3.000000,max_features=3.000000, max_leaf_nodes=3.000000,ccp_alpha=0.500000):
 RMSE: 0.8361341399570807
 MAE: 0.7065174320266894
 R2: -0.00020787582069048405


In [29]:
train_model(500,2,2,2,0)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.000000):
 RMSE: 0.745364275954991
 MAE: 0.617126415409111
 R2: 0.14553484423695584


In [33]:
train_model(500,2,2,2,0.05)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.050000):
 RMSE: 0.7589577803683875
 MAE: 0.637793930034519
 R2: 0.13698058842904837


In [35]:
train_model(500,2,2,2,0.09)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.090000):
 RMSE: 0.770252409457702
 MAE: 0.6439205754756645
 R2: 0.07555019395365137


In [36]:
train_model(500,2,2,2,0.11)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.110000):
 RMSE: 0.7535552390145923
 MAE: 0.6329172203401585
 R2: 0.06441824593079803


In [30]:
train_model(500,2,2,2,0.1)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.100000):
 RMSE: 0.80332607647931
 MAE: 0.6662891121188826
 R2: 0.05437952189958184


In [34]:
train_model(500,2,2,2,0.15)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.150000):
 RMSE: 0.7481096875849403
 MAE: 0.6429998550682055
 R2: 0.0025519432214633486


In [32]:
train_model(500,2,2,2,0.2)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.200000):
 RMSE: 0.8271004975835322
 MAE: 0.698885696413678
 R2: -0.0017502315169521943


In [31]:
train_model(500,2,2,2,0.25)

RandomForestRegressor (n_estimators=500.000000, max_depth=2.000000,max_features=2.000000, max_leaf_nodes=2.000000,ccp_alpha=0.250000):
 RMSE: 0.8733780062505644
 MAE: 0.7441751042535445
 R2: -0.00037920236355515335


In [ ]:
from pyngrok import ngrok
#Terminar túneles abiertos si existen
ngrok.kill()

#Configuración del token de autenticación (opcional)
#Obtén tu token de autenticación https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2XEec4hrhYjLwSGCx8oeWfA5vKl_2Tuwtzko2XNnUL6op3SS8"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

#Abrir un túnel HTTPs en el puerto 5000 para http://localhost:5000
ngrok_tunnel = ngrok.connect(addr ="5000", proto = "http", bind_tls = True)
print("Seguimiento UI MLFlow:", ngrok_tunnel.public_url)

In [ ]:
!mlflow ui